In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Câu 3:**

In [ ]:
#Chỉ định	thư	mục	cần	trích	xuất,	tham	số	N,	import	thư	viện	để	hash	và	trích	xuất N-grams	từ	chuỗi.
from os import listdir
from nltk import ngrams
import hashlib

directories = ["/content/drive/MyDrive/(Lab2)Dataset/Benign PE Samples", "/content/drive/MyDrive/(Lab2)Dataset/Malicious PE Samples"]
N = 2

#Tạo	các	hàm	đọc	tập	tin	và	chuyển	chúng	thành	N-grams.
def read_file(file_path):
    """Reads in the binary sequence of a binary file."""
    with open(file_path, "rb") as binary_file:
        data = binary_file.read()
    return data

def byte_sequence_to_Ngrams(byte_sequence, N):
    """Creates a list of N-grams from a byte sequence."""
    return ngrams(byte_sequence, N)

# Tiến	hành	hash	N-grams.
def hash_input(inp):
 """Compute the MD5 hash of an input."""
 return int(hashlib.md5(inp).hexdigest(), 16)

def make_ngram_hashable(Ngram):
 """Convert N-gram into bytes to be hashable."""
 return bytes(Ngram)

#Hàm	hash_file_Ngrams_into_dictionary	lấy	một	N-grams,	hash	nó,	sau	đó	tăng	số lượng	count	trong	dict	cho	hàm	băm.	Module	B	đảm	bảo	không	thể	có	nhiều	hơn	B	khoá	trong	dict.
 def hash_file_Ngrams_into_dictionary(file_Ngrams, T):
    """Hashes N-grams in a list and then keeps track of the counts in a
dictionary."""
    for Ngram in file_Ngrams:
        hashable_Ngram = make_ngram_hashable(Ngram)
        hashed_and_reduced = hash_input(hashable_Ngram) % B
        T[hashed_and_reduced] = T.get(hashed_and_reduced, 0) + 1

#Giá trị B là số nguyên tố lớn nhất nhỏ hơn 2^16 và tạo dict rỗng. Tiếp theo lặp lại qua các tập tin để count N-grams đã hash.
B = 65521
T = {}
for dataset_path in directories:
    samples = [f for f in listdir(dataset_path)]
    for file in samples:
        file_path = dataset_path + "/" + file
        file_byte_sequence = read_file(file_path)
        file_Ngrams = byte_sequence_to_Ngrams(file_byte_sequence, N)
        hash_file_Ngrams_into_dictionary(file_Ngrams, T)

#Ta chọn 1000 N-gram phổ biến sử dụng với heapq.
K1 = 1000
import heapq

K1_most_common_Ngrams_Using_Hash_Grams = heapq.nlargest(K1, T)

#Sau khi chọn top N-grams được băm, ta tạo bộ thuộc tính N-grams, làm tăng vector đặc trưng.
def featurize_sample(file, K1_most_common_Ngrams_Using_Hash_Grams):
    """Takes a sample and produces a feature vector.
    The features are the counts of the K1 N-grams we've selected.
    """
    K1 = len(K1_most_common_Ngrams_Using_Hash_Grams)
    fv = K1 * [0]
    file_byte_sequence = read_file(file_path)
    file_Ngrams = byte_sequence_to_Ngrams(file_byte_sequence, N)
    for Ngram in file_Ngrams:
        hashable_Ngram = make_ngram_hashable(Ngram)
        hashed_and_reduced = hash_input(hashable_Ngram) % B
        if hashed_and_reduced in  K1_most_common_Ngrams_Using_Hash_Grams:
            index = K1_most_common_Ngrams_Using_Hash_Grams.index(hashed_and_reduced)
            fv[index] += 1
    return fv

#Cuối cùng tạo bộ dữ liệu
X = []
for dataset_path in directories:
    samples = [f for f in listdir(dataset_path)]
    for file in samples:
        file_path = dataset_path + "/" + file
        X.append(featurize_sample(file_path, K1_most_common_Ngrams_Using_Hash_Grams))

#In ra kết quả của vector X
for vector in X:
    print(vector)


[2, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 1, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 

## **Câu 4**

In [12]:
#Do	tập	dữ	liệu	là	các	tập	tin	nhật	ký	định	JSON	nên	import	thư	viện	json
import numpy as np
import os
import json

directories_with_labels = [("/content/drive/MyDrive/(Lab2)Dataset/Benign 1", 0), ("/content/drive/MyDrive/(Lab2)Dataset/Malware 1", 1)]

#Viết	hàm	parse	nhật	ký	JSON.
def get_API_class_method_type_from_log(log):
    """Parses out API calls from behavioral logs."""
    API_data_sequence = []
    with open(log) as log_file:
        json_log = json.load(log_file)
        api_calls_array = "[" + json_log["api_calls"] + "]"
        #Chọn	trích	xuất	class,	method	và	type	từ	API	call.
        api_calls = json.loads(api_calls_array)
        for api_call in api_calls:
            data = api_call["class"] + ":" + api_call["method"] + ":" + api_call["type"]
            API_data_sequence.append(data)
    return API_data_sequence

#Sau	đó	phân	nhãn.
data_corpus = []
labels = []
for directory, label in directories_with_labels:
    logs = os.listdir(directory)
    for log_path in logs:
        file_path = directory + "/" + log_path
        try:
            data_corpus.append(get_API_class_method_type_from_log(file_path))
            labels.append(label)
        except:
            pass
print(data_corpus[0])

#Chia	tập	dữ	liệu	huấn	luyện	và	kiểm	thử
from sklearn.model_selection import train_test_split

corpus_train, corpus_test, y_train, y_test = train_test_split( data_corpus, labels, test_size=0.2, random_state=11 )

#Trích	xuất	N-grams.
import collections
from nltk import ngrams
import numpy as np

def read_file(file_path):
    """Reads in the binary sequence of a binary file."""
    with open(file_path, "rb") as binary_file:
        data = binary_file.read()
    return data

def text_to_Ngrams(text, n):
    """Produces a list of N-grams from a text."""
    Ngrams = ngrams(text, n)
    return list(Ngrams)

def get_Ngram_counts(text, N):
    """Get a frequency count of N-grams in a text."""
    Ngrams = text_to_Ngrams(text, N)
    return collections.Counter(Ngrams)

#Xác	định	N=4	và	thu	thập	tất	cả	N-grams.
N = 4
total_Ngram_count = collections.Counter([])
for file in corpus_train:
    total_Ngram_count += get_Ngram_counts(file, N)

#Sau	đó	thu	hẹp	K1=3000	các	N-grams	phổ	biến.
K1 = 3000
K1_most_frequent_Ngrams = total_Ngram_count.most_common(K1)
K1_most_frequent_Ngrams_list = [x[0] for x in K1_most_frequent_Ngrams]

#Viết	một	hàm	phan	lại	một	mẫu	thành	một	vector	có	N-grams.
def featurize_sample(file, Ngrams_list):
    """Takes a sample and produces a feature vector.
    The features are the counts of the K1 N-grams we've selected.
    """
    K1 = len(Ngrams_list)
    feature_vector = K1 * [0]
    fileNgrams = get_Ngram_counts(file, N)
    for i in range(K1):
        feature_vector[i] = fileNgrams[Ngrams_list[i]]
    return feature_vector

#Sử	dụng	hàm	trên	để	tạo	bộ	huấn	luyện	và	mẫu	kiểm	thử.
X_train = []
for sample in corpus_train:
    X_train.append(featurize_sample(sample,
K1_most_frequent_Ngrams_list))
X_train = np.asarray(X_train)
X_test = []
for sample in corpus_test:
    X_test.append(featurize_sample(sample,
K1_most_frequent_Ngrams_list))
X_test = np.asarray(X_test)

#Tiếp	tục	thu	hẹp	K1=3000	N-12	grams	thành	K2=500.	Sau	đó	thiết	lập	pipeline	chạy	phân	loại	XGBoost.
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

K2 = 500
mi_pipeline = Pipeline(
    [
        ("mutual_information", SelectKBest(mutual_info_classif, k=K2)),
        ("xgb", XGBClassifier()),
    ]
)

#Sau	đó	huấn	luyện	và	kiểm	thử	tập	train,	test.
mi_pipeline.fit(X_train, y_train)
print("Training accuracy:")
print(mi_pipeline.score(X_train, y_train))
print("Testing accuracy:")
print(mi_pipeline.score(X_test, y_test))

['android.content.ContentResolver:registerContentObserver:content', 'libcore.io.IoBridge:open:file', 'android.util.Base64:encode:generic', 'android.accounts.AccountManager:getAccounts:content', 'java.lang.reflect.Method:invoke:reflection', 'android.content.ContentResolver:registerContentObserver:content', 'android.util.Base64:encode:generic', 'libcore.io.IoBridge:open:file', 'android.telephony.TelephonyManager:getNetworkOperatorName:fingerprint', 'java.net.URL:openConnection:network', 'libcore.io.IoBridge:open:file', 'libcore.io.IoBridge:open:file', 'libcore.io.IoBridge:open:file', 'java.lang.reflect.Method:invoke:reflection', 'libcore.io.IoBridge:open:file', 'dalvik.system.DexFile:dalvik.system.DexFile:dex', 'libcore.io.IoBridge:open:file', 'java.lang.reflect.Method:invoke:reflection', 'libcore.io.IoBridge:open:file', 'android.app.ContextImpl:registerReceiver:binder', 'android.app.ContextImpl:registerReceiver:binder', 'android.util.Base64:encodeToString:generic', 'android.os.Debug:isD